# SANS2D: I(Q) workflow for a single run (sample)

This notebook describes in detail the steps that are undertaken in the `sans.to_I_of_Q` workflow.

It assumes the detector data has been recorded in event mode, while the monitor data has been histogrammed.

The data used in this notebook has been published in [Manasi et al. (2021)](#manasi2021),
and we kindly thank the authors for allowing us to use their data.

**Note:** It uses sample run for simplicity and it is not intended to describe complete data reduction pipeline.
The complete pipeline is described in [SANS2D reduction](sans2d_reduction.ipynb).

**Outline:**

- We will begin by loading the data files containing the sample and the direct (empty sample holder) measurements.
- We will then apply some corrections to beamline components specific to the SANS2D beamline.
- This will be followed by some masking of some saturated or defect detector pixels
- Both sample and direct measurement, as well as their monitors, will then be converted to wavelength
- From the direct run, and the direct beam function, the normalization term will be computed
- Both sample measurement and normalization term will be converted to $Q$
- Finally, the sample counts (as a function of $Q$) will be divided by the normalization term (as a function of $Q$)

In [ ]:
import scipp as sc
import esssans as sans
from esssans.types import *
from esssans.logging import configure_workflow
import scippneutron as scn
import sciline

Set up the logging systems of scipp (including scippneutron and ess) and Mantid.

In [ ]:
logger = configure_workflow('sans2d_I_of_Q', filename='sans2d.log')

## Define reduction parameters

We define here whether to include the effects of gravity, and the binning in wavelength and in $Q$ to be used.

In [ ]:
# Include effects of gravity?
gravity = True

params = {}
params[NeXusMonitorName[Incident]] = 'monitor2'
params[NeXusMonitorName[Transmission]] = 'monitor4'
params[WavelengthBins] = sc.linspace(
    dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom'
)
params[QBins] = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')
params[Filename[SampleRun]] = 'sans2d_sample.hdf5'
params[Filename[DirectRun]] = 'sans2d_direct.hdf5'
params[DirectBeamFilename] = 'sans2d_direct_beam.hdf5'
params[NonBackgroundWavelengthRange] = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)

## Loading data files

We load a sample measurement file (`SANS2D00063114.hdf5`) and a direct measurement file (`SANS2D00063091.hdf5`).
We also load the direct beam measurement, which gives a measure of the efficiency of the detector pixels as a function of wavelength.

In [ ]:
pipeline = sciline.Pipeline(sans.providers, params=params)

In [ ]:
pipeline.get(CleanMonitor[SampleRun, Incident]).visualize()

In [ ]:
pipeline.compute(CleanMonitor[SampleRun, Incident]).value

## Inspecting the monitor data

The monitor counts play a major role in normalizing the detector counts.
From the two loaded runs, we extract the data from the incident and transmission monitors.

In [ ]:
monitors = {
    'sample-incident': sample.attrs["monitor2"].value,
    'sample-transmission': sample.attrs["monitor4"].value,
    'direct-incident': direct.attrs["monitor2"].value,
    'direct-transmission': direct.attrs["monitor4"].value,
}

The monitor data contains background noise, which should be removed for best results when normalizing.
Because incident and transmission monitors are located at different distances along the beamline path,
it is more useful to inspect the counts as a function of wavelength instead of time-of-flight.

To compute the wavelength of the neutrons,
we use Scipp's `transform_coords` method
(see [here](https://scipp.github.io/scippneutron/user-guide/coordinate-transformations.html)
for more information about using `transform_coords`).
In order to use `transform_coords`, we need to define a coordinate transformation graph.
The `sans` module provides some pre-defined graphs, one of which is applicable for monitors.

In [ ]:
monitor_graph = sans.conversions.sans_monitor()
sc.show_graph(monitor_graph, simplified=True)

It is then trivial to convert the monitor data to wavelength using

In [ ]:
monitors = {
    key: mon.transform_coords('wavelength', graph=monitor_graph)
    for key, mon in monitors.items()
}

We now plot them on the same figure to asses the level of background noise

In [ ]:
sc.plot(monitors, norm='log', grid=True)

From this, we define a wavelength range between 0.7 &#8491; and 17.1 &#8491; where data is not considered to be background.

Then, we subtract a mean background value from each monitor,
and rebin the data to the wavelength binning defined at the start of the notebook.

In [ ]:
processed_monitors = sans.i_of_q.preprocess_monitor_data(
    monitors, non_background_range=non_background_range, wavelength_bins=wavelength_bins
)

sample_monitors = {
    'incident': processed_monitors['sample-incident'],
    'transmission': processed_monitors['sample-transmission'],
}
direct_monitors = {
    'incident': processed_monitors['direct-incident'],
    'transmission': processed_monitors['direct-transmission'],
}

## Masking bad detector pixels

**Note:** We use programmatic masks here and not those stored in xml files.

Now that the monitor data is cleaned and binned to the correct wavelength range, we turn to the detector data.
The first step is to mask noisy and saturated pixels.
We mask the edges of the square-shaped detector panel with a simple distance relation.
We also mask the region close to the beam center,
where the sample holder is visible as a dark patch with an arm extending to the north-east.

In [ ]:
mask_edges = (
    sc.abs(sample.coords['position'].fields.x) > sc.scalar(0.48, unit='m')
) | (sc.abs(sample.coords['position'].fields.y) > sc.scalar(0.45, unit='m'))

summed = sample.sum('tof')
holder_mask = (
    (summed.data < sc.scalar(100, unit='counts'))
    & (sample.coords['position'].fields.x > sc.scalar(0, unit='m'))
    & (sample.coords['position'].fields.x < sc.scalar(0.42, unit='m'))
    & (sample.coords['position'].fields.y < sc.scalar(0.05, unit='m'))
    & (sample.coords['position'].fields.y > sc.scalar(-0.15, unit='m'))
)

sample.masks['edges'] = mask_edges
sample.masks['holder_mask'] = holder_mask

A good sanity check is to view the masks on the instrument view:

In [ ]:
scn.instrument_view(sample.hist(), pixel_size=0.0075)

## Beam center finder

The beam is not guaranteed to travel through the center of the detector panel,
and we thus have to apply a horizontal and vertical offset to our pixel positions so that the beam centre is at `x = y = 0`.
This is necessary for subsequent azimuthal averaging of the data counts into $Q$ bins.

The `beam_center` utility in the `sans` module is designed for this.
It requires us to define a $Q$ range over which convergence will be checked.

In [ ]:
q_range = sc.linspace('Q', 0.02, 0.3, 71, unit='1/angstrom')

center = sans.beam_center(
    data=sample,
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    wavelength_bins=wavelength_bins,
    q_bins=q_range,
    gravity=gravity,
)
print(center)

# Now shift pixels positions to get the correct beam center
sample.coords['position'] -= center

We once again use the instrument view to verify the beam center has been found and applied correctly.

In [ ]:
scn.instrument_view(sample.hist(), pixel_size=0.0075)

## Coordinate transformation graph

To compute the wavelength $\lambda$, the scattering angle $\theta$, and the $Q$ vector for our data,
we construct a coordinate transformation graph.

It is based on classical conversions from `tof` and pixel `position` to $\lambda$ (`wavelength`),
$\theta$ (`theta`) and $Q$ (`Q`),
but takes into account the Earth's gravitational field, which bends the flight path of the neutrons,
to compute the scattering angle $\theta$.

The angle can be found using the following expression ([Seeger & Hjelm 1991](#seeger1991))

$$\theta = \frac{1}{2}\sin^{-1}\left(\frac{\sqrt{ x^{2} + \left( y + \frac{g m_{\rm n}}{2 h^{2}} \lambda^{2} L_{2}^{2} \right)^{2} } }{L_{2}}\right)$$

where $x$ and $y$ are the spatial coordinates of the pixels in the horizontal and vertical directions, respectively,
$m_{\rm n}$ is the neutron mass,
$L_{2}$ is the distance between the sample and a detector pixel,
$g$ is the acceleration due to gravity,
and $h$ is Planck's constant.

The monitors require a slightly different conversion graph,
as the flight path of the neutrons hitting them does not scatter through the sample,
it links the source to the monitor with a straight line.

The conversion graphs for the detectors and the monitors are defined in the `sans` module,
and can be obtained via

In [ ]:
graph = sans.conversions.sans_elastic(gravity=gravity)

sc.show_graph(graph, simplified=True)

## Convert the data to wavelength

To convert the sample data to wavelength,
we use once again the `transform_coords` utility.

In [ ]:
sample = sample.transform_coords("wavelength", graph=graph)
sample

## Mask Bragg edges in wavelength

We will now take out the wavelength regions with Bragg peaks from the beam stop and detector window,
although in reality the peaks appear only close to the beam stop,
and will make little difference to $I(Q)$.

This could be implemented as masking specific wavelength bins for a specific region in space,
but for now we keep it simple.

In [ ]:
mask = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[True]),
    coords={
        'wavelength': sc.array(
            dims=['wavelength'], values=[2.21, 2.59], unit='angstrom'
        )
    },
)
masking_args = dict(mask=mask, name='bragg_peaks')
sample = sans.common.mask_range(sample.bin(wavelength=1), **masking_args)

sample.hist(wavelength=200).sum('spectrum').plot()

### Adding wavelength masks the monitor data

Because the monitor data is histogrammed (it does not contain events),
the wavelength masking is applied slightly differently.

In [ ]:
sample_monitors = {
    key: sans.common.mask_range(mon, **masking_args)
    for key, mon in sample_monitors.items()
}
direct_monitors = {
    key: sans.common.mask_range(mon, **masking_args)
    for key, mon in direct_monitors.items()
}

## Compute normalization term

In a SANS experiment, the scattering cross section $I(Q)$ is defined as ([Heenan et al. 1997](#heenan1997))

$$ I(Q) = \frac{\partial\Sigma{Q}}{\partial\Omega} = \frac{A_{H} \Sigma_{R,\lambda\subset Q} C(R, \lambda)}{A_{M} t \Sigma_{R,\lambda\subset Q}M(\lambda)T(\lambda)D(\lambda)\Omega(R)} $$

where $A_{H}$ is the area of a mask (which avoids saturating the detector) placed between the monitor of area $A_{M}$ and the main detector.
$\Omega$ is the detector solid angle, and $C$ is the count rate on the main detector, which depends on the position $R$ and the wavelength.
$t$ is the sample thickness, $M$ represents the incident monitor count rate, and $T$ is known as the transmission fraction.
Finally, $D$ is the 'direct beam function', and is defined as

$$ D(\lambda) = \frac{\eta(\lambda)}{\eta_{M}(\lambda)} \frac{A_{H}}{A_{M}} $$

where $\eta$ and $\eta_{M}$ are the detector and monitor efficiencies, respectively.

Hence, in order to normalize the main detector counts $C$, we need compute the transmission fraction $T(\lambda)$,
the direct beam function $D(\lambda)$ and the solid angle $\Omega(R)$.

### Transmission fraction

The transmission fraction is computed from the monitor counts.
It essentially compares the neutron counts before the sample, and after the sample,
to give an absorption profile of the sample as a function of wavelength.

It is defined as the ratio of counts between on the transmission monitor to the counts on the incident monitor for the sample run,
multiplied by the inverse ratio for the direct run, i.e.

$$ T(\lambda) = \frac{M_{\rm sample}^{\rm transmission}}{M_{\rm sample}^{\rm incident}} \frac{M_{\rm direct}^{\rm incident}}{M_{\rm direct}^{\rm transmission}} $$

The transmission fraction is finally computed by using 

In [ ]:
transmission_fraction = sans.normalization.transmission_fraction(
    data_monitors=sample_monitors, direct_monitors=direct_monitors
)
transmission_fraction

In [ ]:
transmission_fraction.plot()

### Direct beam function

The direct beam function is a parameter of the instrument that is well-known to the instrument scientist,
and does not vary much over time.
It is usually stored in a file, and updated a small number of times per year.

Here, we use the direct beam function for the SANS2D instrument from file (loaded at the top of the notebook),
and perform an interpolation so that it spans the same wavelength range as the one requested at the top of the notebook.

In [ ]:
direct_beam = sans.i_of_q.resample_direct_beam(
    direct_beam=direct_beam, wavelength_bins=wavelength_bins
)

sc.plot(direct_beam)

### The denominator term

We combine all the terms above to compute the `denominator`.

**Note:**

[Heybrock et al. (2023)](#heybrock2023) describe how broadcasting operations introduce correlations which are not tracked by Scipp's uncertainty propagation.
In the normalization term above, multiplying $M(\lambda)T(\lambda)D(\lambda)$ (wavelength dependent) by the solid angle $\Omega(R)$ (pixel dependent) is such a broadcasting operation.
The article however states that for normalization operations, under the limit where the counts of the numerator a much smaller than that of the denominator,
dropping the variances of the denominator is a satisfactory approximation.

The helper function below internally calculates the solid angles of the detector pixels $\Omega(R)$,
and performs a verification that our data satisfies that condition.

We are not able to perform this check earlier in the workflow, because the verification involves comparing integrated counts,
and the integration needs to be performed over the same wavelength range for both the monitors and the detector counts.
To be able to compute the wavelengths of the detector data, we needed to first run the beam center finder.

In [ ]:
denominator = sans.normalization.iofq_denominator(
    data=sample,
    data_transmission_monitor=sample_monitors['transmission'],
    direct_incident_monitor=direct_monitors['incident'],
    direct_transmission_monitor=direct_monitors['transmission'],
    direct_beam=direct_beam,
)

# Insert a copy of coords needed for conversion to Q.
# TODO: can this be avoided by copying the Q coords from the converted numerator?
for coord in ['position', 'sample_position', 'source_position']:
    denominator.coords[coord] = sample.meta[coord]

denominator

In [ ]:
sc.plot(denominator.sum('spectrum'), norm='log')

## Convert to Q

Using the coordinate transformation graph as above,
we can compute the momentum vector $Q$, and then merge all the events in the detector pixel bins,
so as to obtain an intensity that depends only on $Q$.

This is done with the `convert_to_q_and_merge_spectra` helper.

In [ ]:
wavelength_bands = sc.concat(
    [wavelength_bins.min(), wavelength_bins.max()], dim='wavelength'
)

In [ ]:
sample_q = sans.i_of_q.convert_to_q_and_merge_spectra(
    data=sample,
    graph=graph,
    wavelength_bands=wavelength_bands,
    q_bins=q_bins,
    gravity=gravity,
)

In [ ]:
sc.plot(sample_q.hist(), norm='log')

### Convert denominator to Q

Converting the denominator to $Q$ is achieved in the same way

In [ ]:
denominator_q = sans.i_of_q.convert_to_q_and_merge_spectra(
    data=denominator,
    graph=graph,
    wavelength_bands=wavelength_bands,
    q_bins=q_bins,
    gravity=gravity,
)

sc.plot(denominator_q, norm='log')

## Normalize the sample

Finally, we normalize the sample with the denominator as a function of $Q$.

In [ ]:
sample_normalized = sans.normalization.normalize(
    numerator=sample_q, denominator=denominator_q
)
sample_normalized

In [ ]:
sc.plot(sample_normalized.hist())

## References

<div id='heenan1997'></div>

Heenan R. K., Penfold J., King S. M., **1997**,
*SANS at Pulsed Neutron Sources: Present and Future Prospects*,
[J. Appl. Cryst., 30, 1140-1147](https://doi.org/10.1107/S0021889897002173)

<div id='heybrock2023'></div>

Heybrock S., Wynen J.-L., Vaytet N., **2023**,
*Systematic underestimation of uncertainties by widespread neutron-scattering data-reduction software*,
Journal of Neutron Research

<div id='manasi2021'></div>

Manasi I., Andalibi M. R., Atri R. S., Hooton J., King S. M., Edler K. J., **2021**,
*Self-assembly of ionic and non-ionic surfactants in type IV cerium nitrate and urea based deep eutectic solvent*,
[J. Chem. Phys. 155, 084902](https://doi.org/10.1063/5.0059238)

<div id='seeger1991'></div>

Seeger P. A., Hjelm R. P. Jnr, **1991**,
*Small-angle neutron scattering at pulsed spallation sources*,
[J. Appl. Cryst., 24, 467-478](https://doi.org/10.1107/S0021889891004764)